# 模型融合预测与交易决策引擎

---

### **目标**
本 Notebook 是整个项目的**应用终端**。它的核心职责是模拟实盘环境，对指定的股票执行完整的“数据-预测-决策”流程。

### **工作流程**
1.  **环境设置与目标选定**: 导入库，加载配置，并指定今天要进行预测的目标股票。
2.  **加载已训练构件**: 
    - 自动查找并加载目标股票**最新版本**的 LGBM 和 LSTM 模型。
    - 加载对应的 `StandardScaler`。
    - 加载两个模型完整的 IC 历史记录，用于动态权重计算。
3.  **获取最新特征**: 调用数据处理流水线，获取截至**今天**的最新一行特征数据。
4.  **独立模型预测**: 分别使用 LGBM 和 LSTM 模型对最新特征进行预测。
5.  **动态权重融合**: 
    - 基于 IC 历史计算出 LGBM 和 LSTM 当前的动态权重。
    - 对两个模型的预测值（经过 Z-score 标准化）进行加权融合，得到最终的预测信号。
6.  **风险审批与决策输出**: 
    - 将融合后的信号提交给 `RiskManager` 进行最终审批（如重复信号检查）。
    - 根据审批结果，输出明确的交易决策（如：**【批准开仓：买入】** 或 **【信号被拒：重复信号】**）。

## 1. 环境设置与目标选定

In [ ]:
import sys
import json
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import torch
import joblib
from tqdm.autonotebook import tqdm
from sklearn.preprocessing import StandardScaler
from utils.config_utils import load_and_merge_configs_for_notebook

# --- 1. 环境与路径设置 ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    print(f"将项目根目录添加到 sys.path: {project_root}")
    sys.path.append(project_root)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# --- 2.  加载配置和所有模块 ---
# a. 加载配置
config = load_and_merge_configs_for_notebook()

# b. 手动加载所有模块
print("--- 正在加载所有项目模块... ---")
try:
    from main_train import run_single_stock_prediction
    from data_process.get_data import initialize_apis, shutdown_apis, get_full_feature_df, get_latest_global_data
    from data_process.save_data import run_data_pipeline, get_processed_data_path
    from model.build_models import run_training_for_ticker
    from utils.hpo_utils import run_hpo_for_ticker
    from model.builders.model_fuser import ModelFuser
    from model.builders.lgbm_builder import LGBMBuilder
    from model.builders.lstm_builder import LSTMBuilder, LSTMModel
    from model.builders.tabtransformer_builder import TabTransformerBuilder, TabTransformerModel
    from risk_management.risk_manager import RiskManager
    from utils.date_utils import resolve_data_pipeline_dates
    from utils.encoding_utils import encode_categorical_features
    from utils.file_utils import find_latest_artifact_paths
    from utils.ml_utils import walk_forward_split
    print("INFO: 项目模块导入成功。")
except ImportError as e:
    raise ImportError(f"模块导入失败，请确保 Notebook 的运行目录在项目根目录。错误: {e}")

# c. 构建 modules 字典
modules = {
    'initialize_apis': initialize_apis, 'shutdown_apis': shutdown_apis,
    'get_full_feature_df': get_full_feature_df, 'get_latest_global_data': get_latest_global_data,
    'run_data_pipeline': run_data_pipeline, 'get_processed_data_path': get_processed_data_path,
    'run_training_for_ticker': run_training_for_ticker, 'run_hpo_for_ticker': run_hpo_for_ticker,
    'ModelFuser': ModelFuser, 'LGBMBuilder': LGBMBuilder, 'LSTMBuilder': LSTMBuilder, 'LSTMModel': LSTMModel,
    'TabTransformerBuilder': TabTransformerBuilder, 'TabTransformerModel': TabTransformerModel,
    'RiskManager': RiskManager,
    'resolve_data_pipeline_dates': resolve_data_pipeline_dates,
    'encode_categorical_features': encode_categorical_features,
    'find_latest_artifact_paths': find_latest_artifact_paths,
    'walk_forward_split': walk_forward_split,
    'pd': pd, 'torch': torch, 'joblib': joblib, 'tqdm': tqdm, 'StandardScaler': StandardScaler, 'Path': Path, 'yaml': yaml, 'json': json
}

# 预测流程

In [ ]:
# 可指定预测股票，如果留空，则会使用 config.yaml 中 application_settings.prophet_target_ticker 定义的值
TARGET_TICKER_OVERRIDE = '000100.SZ' # 例如: '000100.SZ'

if config and modules:
    try:
        run_single_stock_prediction(
            config=config, 
            modules=modules, 
            target_ticker=TARGET_TICKER_OVERRIDE
        )
    except Exception as e:
        print(f"\n--- ERROR: 在执行单点预测时发生严重错误 ---")
        print(e)
        import traceback
        traceback.print_exc()